# Frac Production Data Cleaning - Ohio

In [1]:
# Necessary imports
import numpy as np
import pandas as pd
# Cleaning import
import re

## Ohio Production Data

In [2]:
# Read in Ohio wells data
df = pd.read_csv('OH/HPDIHeader OHIO.csv')
df.head()

,ENTITY_ID,CMN,DISTRICT,PDEN_TYPE,PROD_TYPE,STATE,COUNTRY,PDEN_NAME,API_NO,COMP_DATE,...,SUM6_WTR,SUM9_WTR,AVG3_GAS,AVG6_GAS,AVG9_GAS,AVG3_LIQ,AVG6_LIQ,AVG9_LIQ,AVG3_WTR,AVG6_WTR
0,111656609,1,(N/A),WELL,O&G,OH,USA,FEWELL,34-065-20133-01-00,03-28-1999,...,0,0,0,0,0,0,0,0,0,0
1,111568044,1,(N/A),WELL,O&G,OH,USA,LEDCO (LAWRENCE ECONOMIC DEVELOPMENT),34-087-20645-00-00,01-16-2007,...,0,0,306,358,342,0,0,0,0,0
2,111700768,1,(N/A),WELL,OTHER,OH,USA,CITY OF AKRON (SMP #4),34-153-22968-00-00,09-18-2006,...,0,0,0,0,0,0,0,0,0,0
3,111698888,1,(N/A),WELL,O&G,OH,USA,GREEN UNIT,34-151-25157-00-00,10-25-1999,...,0,0,8926,11112,10779,98,127,124,0,0
4,111648264,1,(N/A),WELL,DSP,OH,USA,SUPREME COUNCIL-HOUSE OF JACOBS,34-031-25921-00-00,12-22-1986,...,0,0,969,1154,1099,229,272,259,0,0


Since there are too many columns to look at at once, print the column name and contents for one well to determine what information is in the data frame

In [3]:
for col in df.columns:
    print(col, '\t\t', df.loc[3,col])

ENTITY_ID 		 111698888
CMN 		 1
DISTRICT 		 (N/A)
PDEN_TYPE 		 WELL
PROD_TYPE 		 O&G
STATE 		 OH
COUNTRY 		 USA
PDEN_NAME 		 GREEN UNIT
API_NO 		 34-151-25157-00-00
COMP_DATE 		 10-25-1999
OFFSHORE 		 nan
FIELD 		 (N/A)
RESERVOIR 		 ROSE RUN SANDSTONE
LEASE_NO 		 5157.0
COMMINGLE_NO 		 0
COMMINGLE_DATE 		 nan
COUNTY_ID 		 34151
COUNTY 		 STARK (OH)
WELL_NO 		 1-KH
CURR_OPER_ID 		 100151646
CURR_OPER_NO 		 7578
CURR_OPER_NAME 		 ENERVEST OPERATING, LLC
LIQ_GATH_NAME_1 		 (N/A)
GAS_GATH_NAME_1 		 (N/A)
STATUS 		 ACTIVE
DRILL_TYPE 		 H
ELEVATION 		 1108
ELEVATION_TYPE 		 GRD
TOTAL_DEPTH 		 7525
UPPER_PERF 		 7345
LOWER_PERF 		 7397
LIQ_GRAV 		 0
GAS_GRAV 		 0
LIQ_DAILY 		 0
GAS_DAILY 		 4
LIQ_CUM 		 1991
GAS_CUM 		 222179
LATITUDE 		 40.968560700000005
LONGITUDE 		 -81.2102456
LOC_REMARK 		 nan
FIRST_PROD_DATE 		 11-01-1999
LAST_PROD_DATE 		 12-01-2017
LATEST_LIQ 		 0
LATEST_GAS 		 122
LATEST_WTR 		 0
LATEST_WCNT 		 1
PRIOR12_LIQ 		 0
PRIOR12_GAS 		 1328
PRIOR12_WTR 		 0
FIRST_LIQ 		 87
F

Columns of interest:
- API_NO is the API number - I will need to __clean the dashes out__
- DRILL_TYPE - I will need to make sure and __keep the horizontal (H) wells only__
- TOTAL_DEPTH - horizontal + vertical depth
- UPPER_PERF and LOWER_PERF to calculate gross perforated interval
- FORMATION?
- FIRST6_BOE
- ELEVATION

I'm missing the horizontal length, which I will need to __calculate from the differences in longitude and latitude from the top and bottom of the well__.
According to [sciencing](https://sciencing.com/convert-latitude-longtitude-feet-2724.html), to calculate length from longitude and latitude, take the difference in longitude or latitide in degrees (which is given), and use the following equation: $ Dist_{prime meridian} = Degrees * \frac{10,000km}{90 degrees} * 3280.4 \frac{ft}{km} $
After calculating each length, then I will need to calculate the euclidian distance between the two.

In [4]:
# Pulling out relevant columns
ohwells_prod = df[['API_NO','DRILL_TYPE','TOTAL_DEPTH', 'ELEVATION',
               'UPPER_PERF','LOWER_PERF','FORMATION','FIRST6_BOE',
              'LATITUDE_BOTM', 'LONGITUDE_BOTM','LATITUDE_EX1',
               'LONGITUDE_EX1']]
ohwells_prod.head()

,API_NO,DRILL_TYPE,TOTAL_DEPTH,ELEVATION,UPPER_PERF,LOWER_PERF,FORMATION,FIRST6_BOE,LATITUDE_BOTM,LONGITUDE_BOTM,LATITUDE_EX1,LONGITUDE_EX1
0,34-065-20133-01-00,H,2928,925,0,0,TRENTON LIMESTONE,0,40.751689,-83.541598,40.747182,-83.540479
1,34-087-20645-00-00,H,2186,562,0,0,OLENTANGY SHALE,306,38.429559,-82.591706,38.427768,-82.587812
2,34-153-22968-00-00,H,4410,961,0,0,SALINA GROUP,0,41.020258,-81.561900,41.017940,-81.558187
3,34-151-25157-00-00,H,7525,1108,7345,7397,ROSE RUN SANDSTONE,10052,40.968062,-81.207431,40.968619,-81.210070
4,34-031-25921-00-00,H,3907,1020,0,0,QUEENSTON FORMATION,2377,0.000000,0.000000,40.318357,-81.904299


Now that I've removed the non-relevant columns, let's get to work engineering the same features we used for Colorado wells, Gross Perforated Interval, and keeping only horizontal drill types.

In [5]:
# Cleaning the API number and setting it as the index
ohwells_prod.index = ohwells_prod.API_NO.map(lambda x: re.sub('-', '', x))

# Calculating Gross Perforated Interval
ohwells_prod['GrossPerforatedInterval'] = ohwells_prod.LOWER_PERF - ohwells_prod.UPPER_PERF

# Selecting only horizontal wells
ohwells_prod = ohwells_prod[ohwells_prod.DRILL_TYPE == 'H']

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Now, let's calculate the horizontal length, using the equation above to convert the difference in logitude and latitude to triangulate the horizontal length of the well.

In [6]:
# Calculating the differences in degrees
long_diff = ohwells_prod.LONGITUDE_BOTM - ohwells_prod.LONGITUDE_EX1
lat_diff = ohwells_prod.LATITUDE_BOTM - ohwells_prod.LATITUDE_EX1

# Calculating the distances in longitude and latitude
long_dist = long_diff * 10000 / 90 * 3280.4
lat_dist = lat_diff * 10000 / 90 * 3280.4

# Calculating the distance between the two lengths and setting that equal
# to the horizontal length of the well
ohwells_prod['HorizontalLength'] = np.sqrt(lat_dist**2 + long_dist**2)

Great, except I noticed that some wells didn't have two values for longitude and latitude.  This would produce an extremely large horizontal length, so I will filter out only the relevant lengths, and calculate the vertical depth from wells that did not provide that information.

In [7]:
# Selecting only wells with reasonable horizontal lengths
ohwells_prod = ohwells_prod[ohwells_prod['HorizontalLength']<1e6]

# Calculating vertical depth from total depth and horizontal length
ohwells_prod['VerticalDepth'] = ohwells_prod.TOTAL_DEPTH - ohwells_prod.HorizontalLength

# Selecting only wells with positive production in the first 6 months
ohwells_prod = ohwells_prod[ohwells_prod.FIRST6_BOE > 0]

In [8]:
# Checking the shape of the producing wells
ohwells_prod.shape

(1788, 15)

Fantastic! Now, it's time to join this data with the fracfocus data that has the types of frac methods.

## FracFocus Data

In [33]:
# Read in fracfocus data
fracfocus = pd.read_csv('fracfocus_clean.csv', index_col=0)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


I only want to keep the engineered columns of the frac types and the mass of sand, so I will isolate those.

In [10]:
fracfocus = fracfocus[['slick','gel','xlinkgel','sandmass']]

Now, let's join the two data frames on the API number.

In [11]:
ohfracwells = ohwells_prod.join(fracfocus, how='left')
print(ohfracwells.shape)
ohfracwells.head()

(1788, 19)


,API_NO,DRILL_TYPE,TOTAL_DEPTH,ELEVATION,UPPER_PERF,LOWER_PERF,FORMATION,FIRST6_BOE,LATITUDE_BOTM,LONGITUDE_BOTM,LATITUDE_EX1,LONGITUDE_EX1,GrossPerforatedInterval,HorizontalLength,VerticalDepth,slick,gel,xlinkgel,sandmass
API_NO,,,,,,,,,,,,,,,,,,,
34087206450000,34-087-20645-00-00,H,2186,562,0,0,OLENTANGY SHALE,306,38.429559,-82.591706,38.427768,-82.587812,0,1562.285474,623.714526,NaN,NaN,NaN,NaN
34151251570000,34-151-25157-00-00,H,7525,1108,7345,7397,ROSE RUN SANDSTONE,10052,40.968062,-81.207431,40.968619,-81.210070,52,983.274953,6541.725047,NaN,NaN,NaN,NaN
34079202480000,34-079-20248-00-00,H,2810,860,2694,2699,OHIO SHALE,63,38.929290,-82.635120,38.922716,-82.626506,5,3949.804160,-1139.804160,NaN,NaN,NaN,NaN
34151251170000,34-151-25117-00-00,H,6640,985,6510,6591,ROSE RUN SANDSTONE,19781,40.686860,-81.579932,40.687908,-81.581567,81,707.872576,5932.127424,NaN,NaN,NaN,NaN
34065201330000,34-065-20133-00-00,H,2928,925,0,0,PRECAMBRIAN,51,40.750120,-83.540834,40.751741,-83.541531,0,643.225647,2284.774353,NaN,NaN,NaN,NaN


It doesn't look lke there was any overlap in the data frame API numbers. Let me check if any did populate.

In [12]:
ohfracwells.notnull().sum()

API_NO                     1788
DRILL_TYPE                 1788
TOTAL_DEPTH                1788
ELEVATION                  1788
UPPER_PERF                 1788
LOWER_PERF                 1788
FORMATION                  1788
FIRST6_BOE                 1788
LATITUDE_BOTM              1788
LONGITUDE_BOTM             1788
LATITUDE_EX1               1788
LONGITUDE_EX1              1788
GrossPerforatedInterval    1788
HorizontalLength           1788
VerticalDepth              1788
slick                         0
gel                           0
xlinkgel                      0
sandmass                      0
dtype: int64

Nope...I'll try the other data frame, fracmethod that just has the frac method information

## Frac Method Joining

In [36]:
# Read in the frac methods
fracmethod = pd.read_csv('fracmethod_clean.csv', index_col=0)
fracmethod.head()

,slick,gel,xlinkgel,sandmass
APINumber,,,,
4226932868,1,0,1,8552.338261
1007203590000,1,1,0,177493.467769
1007203600000,1,1,0,151188.848979
1007203610000,1,1,0,219070.601388
1007203620000,1,1,0,245962.571934


Now, let's try to join the two together.

In [38]:
ohfracwells_prod = ohwells_prod.join(fracmethod, how='left')
print(ohfracwells_prod.shape)
ohfracwells_prod.notnull().sum()

(1788, 19)


API_NO                     1788
DRILL_TYPE                 1788
TOTAL_DEPTH                1788
ELEVATION                  1788
UPPER_PERF                 1788
LOWER_PERF                 1788
FORMATION                  1788
FIRST6_BOE                 1788
LATITUDE_BOTM              1788
LONGITUDE_BOTM             1788
LATITUDE_EX1               1788
LONGITUDE_EX1              1788
GrossPerforatedInterval    1788
HorizontalLength           1788
VerticalDepth              1788
slick                         0
gel                           0
xlinkgel                      0
sandmass                      0
dtype: int64

Nope...that didn't work either.  After spot checking, I can't find any of the API numbers from this data frame in the frac focus data, but there are definitely Ohio wells in the frac focus data.  Let me try the last data file to see what it has.

## Well Spots Ohio Data

In [39]:
df2 = pd.read_csv('OH/HPDIWellSpots OHIO.csv')
print(df2.shape)
df2.head()

(3259, 29)


,ENTITY_ID,SPOT_ID,STATE,ALL_API_NO,WELL_NAME,WELL_NO,LATITUDE,LONGITUDE,LATITUDE_EX1,LONGITUDE_EX1,...,TOTAL_DEPTH,PLUG_DATE,COMP_DATE,LAST_OPER_NAME,ABSTRACT,SURVEY,TX_BLOCK,TX_SEC,SUBSURVEY,TX_SCRAP
0,111656609,111656609,OH,34-065-20133-01-00,FEWELL,1-25,40.747128,-83.540552,40.747182,-83.540479,...,2928,NaN,NaN,KNOX ENERGY INC.,NaN,NaN,NaN,NaN,NaN,NaN
1,111568044,111568044,OH,34-087-20645-00-00,LEDCO (LAWRENCE ECONOMIC DEVELOPMENT),1,38.427687,-82.587930,38.427768,-82.587812,...,2186,NaN,NaN,NYTIS EXPLORATION CO LLC,NaN,NaN,NaN,NaN,NaN,NaN
2,111700768,111700768,OH,34-153-22968-00-00,CITY OF AKRON (SMP #4),75,41.017885,-81.558346,41.017940,-81.558187,...,4410,NaN,NaN,CARGILL INC,NaN,NaN,NaN,NaN,NaN,NaN
3,111698888,111698888,OH,34-151-25157-00-00,GREEN UNIT,1-KH,40.968561,-81.210246,40.968619,-81.210070,...,7525,NaN,NaN,ENERVEST OPERATING LLC,NaN,NaN,NaN,NaN,NaN,NaN
4,111648264,111648264,OH,34-031-25921-00-00,SUPREME COUNCIL-HOUSE OF JACOBS,1,40.318239,-81.904274,40.318357,-81.904299,...,3907,NaN,NaN,NGO DEVELOPMENT CORP.,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df2.columns

Index(['ENTITY_ID', 'SPOT_ID', 'STATE', 'ALL_API_NO', 'WELL_NAME', 'WELL_NO',
       'LATITUDE', 'LONGITUDE', 'LATITUDE_EX1', 'LONGITUDE_EX1', 'SYMBOL',
       'RELIABILITY', 'UPPER_PERF', 'LOWER_PERF', 'PROD_TYPE', 'STATUS',
       'DRILL_TYPE', 'ELEVATION', 'ELEVATION_TYPE', 'TOTAL_DEPTH', 'PLUG_DATE',
       'COMP_DATE', 'LAST_OPER_NAME', 'ABSTRACT', 'SURVEY', 'TX_BLOCK',
       'TX_SEC', 'SUBSURVEY', 'TX_SCRAP'],
      dtype='object')

It looks like it has most of the same columns I need, except for the production, which is my target variable.  I might be able to join this with the previous Ohio data frame?

I will perform the same analysis as above on this data frame.

In [41]:
ohwells = df2[['ALL_API_NO','UPPER_PERF','LOWER_PERF','PROD_TYPE',
              'DRILL_TYPE','ELEVATION','TOTAL_DEPTH','LATITUDE_EX1', 'LATITUDE', 
              'LONGITUDE_EX1','LONGITUDE']]

In [42]:
ohwells.head()

,ALL_API_NO,UPPER_PERF,LOWER_PERF,PROD_TYPE,DRILL_TYPE,ELEVATION,TOTAL_DEPTH,LATITUDE_EX1,LATITUDE,LONGITUDE_EX1,LONGITUDE
0,34-065-20133-01-00,0,0,O&G,H,925,2928,40.747182,40.747128,-83.540479,-83.540552
1,34-087-20645-00-00,0,0,O&G,H,562,2186,38.427768,38.427687,-82.587812,-82.587930
2,34-153-22968-00-00,0,0,OTHER,H,961,4410,41.017940,41.017885,-81.558187,-81.558346
3,34-151-25157-00-00,7345,7397,O&G,H,1108,7525,40.968619,40.968561,-81.210070,-81.210246
4,34-031-25921-00-00,0,0,DSP,H,1020,3907,40.318357,40.318239,-81.904299,-81.904274


In [43]:
# Cleaning the API number and setting it as the index
ohwells.index = ohwells.ALL_API_NO.map(lambda x: re.sub('-', '', x))

# Calculating Gross Perforated Interval
ohwells['GrossPerforatedInterval'] = ohwells.LOWER_PERF - ohwells.UPPER_PERF

# Selecting only horizontal wells
ohwells = ohwells[ohwells.DRILL_TYPE == 'H']

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [44]:
long_diff = ohwells.LONGITUDE - ohwells.LONGITUDE_EX1
lat_diff = ohwells.LATITUDE - ohwells.LATITUDE_EX1

long_dist = long_diff * 10000 / 90 * 3280.4
lat_dist = lat_diff * 10000 / 90 * 3280.4

ohwells['HorizontalLength'] = np.sqrt(lat_dist**2 + long_dist**2)

In [45]:
ohwells = ohwells[ohwells['HorizontalLength']<1e6]

ohwells['VerticalDepth'] = ohwells.TOTAL_DEPTH - ohwells.HorizontalLength

In [46]:
print(ohwells.shape)
ohwells.head()

(3259, 14)


,ALL_API_NO,UPPER_PERF,LOWER_PERF,PROD_TYPE,DRILL_TYPE,ELEVATION,TOTAL_DEPTH,LATITUDE_EX1,LATITUDE,LONGITUDE_EX1,LONGITUDE,GrossPerforatedInterval,HorizontalLength,VerticalDepth
ALL_API_NO,,,,,,,,,,,,,,
34065201330100,34-065-20133-01-00,0,0,O&G,H,925,2928,40.747182,40.747128,-83.540479,-83.540552,0,33.168489,2894.831511
34087206450000,34-087-20645-00-00,0,0,O&G,H,562,2186,38.427768,38.427687,-82.587812,-82.587930,0,52.582623,2133.417377
34153229680000,34-153-22968-00-00,0,0,OTHER,H,961,4410,41.017940,41.017885,-81.558187,-81.558346,0,61.267590,4348.732410
34151251570000,34-151-25157-00-00,7345,7397,O&G,H,1108,7525,40.968619,40.968561,-81.210070,-81.210246,52,67.301361,7457.698639
34031259210000,34-031-25921-00-00,0,0,DSP,H,1020,3907,40.318357,40.318239,-81.904299,-81.904274,0,43.791668,3863.208332


In [47]:
ohwells.PROD_TYPE.value_counts()

O&G        3059
STR         183
OTHER        12
STORAGE       2
DSP           1
GAS           1
SWD           1
Name: PROD_TYPE, dtype: int64

In [48]:
ohfracwells = ohwells.join(fracmethod, how='inner')
print(ohfracwells.shape)
ohfracwells.head()

(0, 18)


,ALL_API_NO,UPPER_PERF,LOWER_PERF,PROD_TYPE,DRILL_TYPE,ELEVATION,TOTAL_DEPTH,LATITUDE_EX1,LATITUDE,LONGITUDE_EX1,LONGITUDE,GrossPerforatedInterval,HorizontalLength,VerticalDepth,slick,gel,xlinkgel,sandmass


In [49]:
ohfracwells = ohwells.join(fracmethod, how='left')
print(ohfracwells.shape)
ohfracwells.head(20)

(3259, 18)


,ALL_API_NO,UPPER_PERF,LOWER_PERF,PROD_TYPE,DRILL_TYPE,ELEVATION,TOTAL_DEPTH,LATITUDE_EX1,LATITUDE,LONGITUDE_EX1,LONGITUDE,GrossPerforatedInterval,HorizontalLength,VerticalDepth,slick,gel,xlinkgel,sandmass
ALL_API_NO,,,,,,,,,,,,,,,,,,
34065201330100,34-065-20133-01-00,0,0,O&G,H,925,2928,40.747182,40.747128,-83.540479,-83.540552,0,33.168489,2894.831511,NaN,NaN,NaN,NaN
34087206450000,34-087-20645-00-00,0,0,O&G,H,562,2186,38.427768,38.427687,-82.587812,-82.587930,0,52.582623,2133.417377,NaN,NaN,NaN,NaN
34153229680000,34-153-22968-00-00,0,0,OTHER,H,961,4410,41.017940,41.017885,-81.558187,-81.558346,0,61.267590,4348.732410,NaN,NaN,NaN,NaN
34151251570000,34-151-25157-00-00,7345,7397,O&G,H,1108,7525,40.968619,40.968561,-81.210070,-81.210246,52,67.301361,7457.698639,NaN,NaN,NaN,NaN
34031259210000,34-031-25921-00-00,0,0,DSP,H,1020,3907,40.318357,40.318239,-81.904299,-81.904274,0,43.791668,3863.208332,NaN,NaN,NaN,NaN
34079202480000,34-079-20248-00-00,2694,2699,O&G,H,860,2810,38.922716,38.922641,-82.626506,-82.626619,5,49.687229,2760.312771,NaN,NaN,NaN,NaN
34119283650000,34-119-28365-00-00,5662,6026,O&G,H,954,6100,40.113904,40.113834,-81.957147,-81.957290,364,58.067004,6041.932996,NaN,NaN,NaN,NaN
34053209870000,34-053-20987-00-00,0,0,O&G,H,600,4196,38.835280,38.835224,-82.226780,-82.226854,0,34.022143,4161.977857,NaN,NaN,NaN,NaN
34151251170000,34-151-25117-00-00,6510,6591,O&G,H,985,6640,40.687908,40.687849,-81.581567,-81.581721,81,60.237929,6579.762071,NaN,NaN,NaN,NaN


In [50]:
ohfracwells.notnull().sum()

ALL_API_NO                 3259
UPPER_PERF                 3259
LOWER_PERF                 3259
PROD_TYPE                  3259
DRILL_TYPE                 3259
ELEVATION                  3259
TOTAL_DEPTH                3259
LATITUDE_EX1               3259
LATITUDE                   3259
LONGITUDE_EX1              3259
LONGITUDE                  3259
GrossPerforatedInterval    3259
HorizontalLength           3259
VerticalDepth              3259
slick                         0
gel                           0
xlinkgel                      0
sandmass                      0
dtype: int64

Nope...I have no idea what's going on here. I'm going to try cleaning another 